# 🚗 Model 2: Demand-Based Dynamic Pricing
This notebook implements a demand-based pricing model for urban parking using:
- Occupancy Rate
- Queue Length
- Vehicle Type
- Traffic Conditions
- Special Events

The model dynamically adjusts prices based on normalized demand.

In [15]:
!pip install pathway bokeh panel --quiet

In [16]:
import pandas as pd
import numpy as np
import datetime
import pathway as pw
import panel as pn
import bokeh.plotting
from google.colab import files

# Upload CSV file
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])

Saving modified_dataset.csv to modified_dataset (2).csv


In [17]:
# Preprocessing
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp').reset_index(drop=True)

# Map weights
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
traffic_weights = {'low': 0.2, 'average': 0.5, 'high': 1.0}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_weights).fillna(1.0)
df['TrafficScore'] = df['TrafficConditionNearby'].map(traffic_weights).fillna(0.5)

# Feature engineering
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df['QueueNorm'] = df['QueueLength'] / df['QueueLength'].max()
df['Demand'] = (
    0.4 * df['OccupancyRate'] +
    0.2 * df['QueueNorm'] +
    0.2 * df['IsSpecialDay'] +
    0.1 * df['VehicleTypeWeight'] -
    0.1 * df['TrafficScore']
)
df['NormalizedDemand'] = (df['Demand'] - df['Demand'].min()) / (df['Demand'].max() - df['Demand'].min())
df['Price'] = 10 * (1 + 1.2 * df['NormalizedDemand'])
df['Price'] = df['Price'].clip(5, 20)
df[['SystemCodeNumber', 'Timestamp', 'Price']].to_csv('pricing_model2.csv', index=False)

In [18]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Price: float

In [19]:
data = pw.demo.replay_csv('pricing_model2.csv', schema=ParkingSchema, input_rate=100)
fmt = "%Y-%m-%d %H:%M:%S"
stream = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)
windowed = stream.windowby(
    pw.this.t,
    instance=pw.this.SystemCodeNumber + '_' + pw.this.day,
    window=pw.temporal.tumbling(datetime.timedelta(days=1)),
    behavior=pw.temporal.exactly_once_behavior()
).reduce(
    t=pw.this._pw_window_end,
    sum_price=pw.reducers.sum(pw.this.Price),
    count_price=pw.reducers.count()
).with_columns(avg_price=pw.this.sum_price / pw.this.count_price)

In [20]:
pn.extension()
def plot_price(source):
    fig = bokeh.plotting.figure(height=400, width=800, x_axis_type='datetime', title='Model 2: Demand-Based Pricing')
    fig.line('t', 'avg_price', source=source, line_width=2, color='green')
    fig.circle('t', 'avg_price', source=source, size=6, color='red')
    return fig
viz = windowed.plot(plot_price, sorting_col='t')
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
%%capture --no-display
pw.run()

Output()